In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Exploration

In [2]:
movies=pd.read_csv("data\movies_metadata.csv")
movies.head()

C:\Users\dhruv\AppData\Local\Temp\ipykernel_22140\3026720149.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv("data\movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
ratings=pd.read_csv("data/ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
#movies

In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
movies.shape

(45466, 24)

In [7]:
movies.isnull().sum().sort_values(ascending=False)

belongs_to_collection    40972
homepage                 37684
tagline                  25054
overview                   954
poster_path                386
runtime                    263
status                      87
release_date                87
imdb_id                     17
original_language           11
spoken_languages             6
title                        6
video                        6
vote_average                 6
revenue                      6
vote_count                   6
popularity                   5
production_companies         3
production_countries         3
original_title               0
id                           0
genres                       0
budget                       0
adult                        0
dtype: int64

In [8]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [9]:
movies['status'].unique()

array(['Released', nan, 'Rumored', 'Post Production', 'In Production',
       'Planned', 'Canceled'], dtype=object)

In [10]:
movies = movies[['id', 'title', 'genres', 'overview', 'tagline', 'release_date', 'vote_average']]

In [11]:
#movies
movies['genres'].unique()

array(["[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",
       ...,
       "[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name': 'Comedy'}, {'id': 28, 'name': 'Action'}, {'id': 10751, 'name': 'Family'}]",
       "[{'id': 28, 'name': 'Action'}, {'id': 9648, 'name': 'Mystery'}, {'id': 53, 'name': 'Thriller'}, {'id': 27, 'name': 'Horror'}]",
       "[{'id': 10751, 'name': 'Family'}, {'id': 16, 'name': 'Animation'}, {'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"],
      dtype=object)

In [12]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies.dropna(subset=['id'], inplace=True)
movies['id'] = movies['id'].astype(int)

In [13]:
movies['genres'] = movies['genres'].fillna('[]')
movies['genres'] = movies['genres'].astype(str)

In [14]:
import ast
import json

def parse_genres(x):
    if isinstance(x, list):
        return [d['name'] for d in x]
    if isinstance(x, str):
        try:
            # Handle malformed strings
            x = x.replace("'", '"')  # replace single quotes with double quotes
            data = json.loads(x)
            return [d['name'] for d in data if 'name' in d]
        except:
            try:
                data = ast.literal_eval(x)
                return [d['name'] for d in data if 'name' in d]
            except:
                return []
    return []

movies['genres'] = movies['genres'].apply(parse_genres)

In [15]:
movies['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [16]:
#Combining OverView with genres for Tfidf 
movies['description'] = (
    movies['overview'].fillna('') + ' ' +
    movies['genres'].apply(lambda x: ' '.join(x))
)

In [17]:
movies['description'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Animation Comedy Family"

In [18]:
from sklearn.neighbors import NearestNeighbors

tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf.fit_transform(movies['description'])  # sparse matrix
nn = NearestNeighbors(metric='cosine', algorithm='brute')
nn.fit(tfidf_matrix)  # for on-demand neighbors

,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [19]:
def get_similar_movies(title, top_n=10):
    idx = movies[movies['title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        return []
    
    idx = idx[0]
    movie_vector = tfidf_matrix[idx]
    cosine_scores = cosine_similarity(movie_vector, tfidf_matrix).flatten()
    
    top_indices = cosine_scores.argsort()[-top_n-1:-1][::-1]
    return movies.iloc[top_indices][['title', 'vote_average', 'genres']]

In [20]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [21]:
movies.to_pickle('movies_clean.pkl')
import pickle
pickle.dump(tfidf, open('tfidf.pkl','wb'))
pickle.dump(nn,   open('nn_tfidf.pkl','wb'))

### Phase 2: Collaborative Filtering

In [22]:
# Merge datasets on movieId
movie_ratings = pd.merge(ratings, movies, left_on='movieId', right_on='id')
movie_ratings.head()

,userId,movieId,rating,timestamp,id,title,genres,overview,tagline,release_date,vote_average,description
0,1,1371,2.5,1260759135,1371,Rocky III,[Drama],"Now the world champion, Rocky Balboa is living...",The greatest challenge.,1982-05-28,6.6,"Now the world champion, Rocky Balboa is living..."
1,1,1405,1.0,1260759203,1405,Greed,"[Drama, History]",Greed is the classic 1924 silent film by Erich...,NaN,1924-12-04,7.5,Greed is the classic 1924 silent film by Erich...
2,1,2105,4.0,1260759139,2105,American Pie,"[Comedy, Romance]","At a high-school party, four friends find that...",There's nothing like your first piece.,1999-07-09,6.4,"At a high-school party, four friends find that..."
3,1,2193,2.0,1260759198,2193,My Tutor,"[Comedy, Drama, Romance]",High school senior Bobby Chrystal fails his Fr...,School's out...But Bobby's education has just ...,1983-03-04,5.8,High school senior Bobby Chrystal fails his Fr...
4,1,2294,2.0,1260759108,2294,Jay and Silent Bob Strike Back,[Comedy],When Jay and Silent Bob learn that their comic...,Hollywood had it coming,2001-08-22,6.4,When Jay and Silent Bob learn that their comic...


In [23]:
# safer pivot using movieId (numeric)
user_movie_matrix_id = movie_ratings.pivot_table(index='userId', columns='movieId', values='rating')


In [24]:
user_movie_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

In [25]:
movie_name = "American Pie"
target_movie_ratings = user_movie_matrix[movie_name]

In [26]:
similar_to_movie = user_movie_matrix.corrwith(target_movie_ratings)
similar_to_movie = similar_to_movie.dropna()

C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [27]:
corr_movie = pd.DataFrame(similar_to_movie, columns=['Correlation'])
corr_movie = corr_movie.sort_values('Correlation', ascending=False)
corr_movie.head()


,Correlation
title,
Trouble in Mind,1.0
"Lock, Stock and Two Smoking Barrels",1.0
The French Revolution,1.0
Cruel Intentions 3,1.0
Sun Alley,1.0


In [28]:
ratings_count = movie_ratings.groupby('title')['rating'].count()
corr_movie = corr_movie.join(ratings_count)
corr_movie = corr_movie[corr_movie['rating'] > 50]  # filter movies with >50 ratings
corr_movie.sort_values('Correlation', ascending=False).head(10)


,Correlation,rating
title,,
X-Men Origins: Wolverine,0.720155,51
The Green Mile,0.637973,60
Shaft in Africa,0.629465,61
Shadows in Paradise,0.612372,59
Belle Époque,0.559297,60
Rain Man,0.545254,198
Rush Hour,0.541577,51
Will Penny,0.537124,72
Heathers,0.518544,75


In [29]:
# quick test - exact title match (case-insensitive)
print(get_similar_movies("Toy Story (1995)", top_n=5))

# test a fuzzy-case (if title names vary)
movie_title = movies['title'].iloc[0]
print("sample title:", movie_title)
print(get_similar_movies(movie_title, top_n=5))

[]
sample title: Toy Story
                        title  vote_average                       genres
15348             Toy Story 3           7.6  [Animation, Family, Comedy]
2997              Toy Story 2           7.3  [Animation, Comedy, Family]
24523               Small Fry           6.8          [Animation, Family]
10301  The 40 Year Old Virgin           6.2            [Comedy, Romance]
8327                The Champ           6.8                      [Drama]


In [30]:
counts = movie_ratings.groupby('movieId').size().sort_values(ascending=False)
top_movie_ids = counts.head(2000).index  # adjust 2000 as memory permits

pivot = movie_ratings[movie_ratings.movieId.isin(top_movie_ids)].pivot_table(index='userId', columns='movieId', values='rating')

def recommend_by_corr(movieId, min_ratings=50, topn=10):
    target = pivot[movieId]
    corrs = pivot.corrwith(target).dropna()
    corrs_df = corrs.to_frame('corr').join(counts.rename('count'))
    corrs_df = corrs_df[corrs_df['count'] >= min_ratings]
    top = corrs_df.sort_values('corr', ascending=False).head(topn+1)  # +1 to exclude self
    return top.index[top.index != movieId][:topn]  # returns movieId list

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_movie_matrix.fillna(0))
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)


In [32]:
with open("user_similarity.pkl", "wb") as f:
    pickle.dump(user_similarity_df, f)

with open("user_movie_matrix.pkl", "wb") as f:
    pickle.dump(user_movie_matrix, f)

In [33]:
movies[['id','title','genres','description','release_date','vote_average']].to_csv('movies_clean.csv', index=False)